# STARSHIP - PART 5

In [1]:
import numpy as np
import pandas as pd

y_train=pd.read_csv('ready data part 4/y_train_data.csv')
X_train=pd.read_csv('ready data part 4/X_train_data.csv')
y_test=pd.read_csv('ready data part 4/y_test_data.csv')
X_test=pd.read_csv('ready data part 4/X_test_data.csv')

## Prepare 2 test set: a. All users with 0 and 1 clicks. B. all users with 2 and more clicks

In [24]:
test_set = pd.concat([X_test, y_test], axis=1)
test_set_a = test_set.loc[test_set['user_clicks']<2] #0-1 clicks
test_set_b = test_set.loc[test_set['user_clicks'] >= 2] 


In [25]:
# change the indexes 0,1,2...
test_set_a = test_set_a.set_index(pd.Index(np.arange(len(test_set_a))))

In [4]:
def evaluate(test_set):
    y_pred = cat_model.predict(test_set)
    print("Accuracy:",metrics.accuracy_score(original_y, y_pred))
    print("Recall:",metrics.recall_score(original_y, y_pred))
    print("F-Measure:",metrics.f1_score(original_y, y_pred))
    print("Precision:",metrics.precision_score(original_y, y_pred))

In [27]:
original_y = test_set_a['is_click']
del test_set_a['is_click']

In [6]:
from sklearn import metrics
from catboost import CatBoostClassifier

cat_model = CatBoostClassifier()
cat_model.fit(X_train  , y_train.values.ravel(),verbose=False)


In [7]:
evaluate(test_set_a)

Accuracy: 0.7889633638506526
Recall: 0.7600862725196651
F-Measure: 0.7362749202130724
Precision: 0.7139101356232638


## For each of the users from data set a find it’s closest neighbour in data set b

In [8]:
from sklearn.metrics import pairwise_distances
from sklearn.metrics import pairwise_distances_argmin
import time
import pickle

def dist_dict(above,metric):
    test_set = pd.concat([X_test, y_test], axis=1)
    #test_set['user_id']= test_set['user_id'].astype("category").cat.codes

    test_set_a = test_set.loc[test_set['user_clicks']<2] #0-1 clicks
    test_set_b = test_set.loc[test_set['user_clicks'] >= above] #more then above
    
    # change the indexes 0,1,2...
    test_set_a = test_set_a.set_index(pd.Index(np.arange(len(test_set_a))))
    test_set_b = test_set_b.set_index(pd.Index(np.arange(len(test_set_b))))
        
    closest_dict = {}
    index = 1000
    indexes_a = test_set_a.index
    indexes_b = test_set_b.index

    
    for i in range(int(len(test_set_a)/index)+1):
        inserted=test_set_a.iloc[i*index:(i+1)*index]
        close_record_index = pairwise_distances_argmin(inserted,test_set_b,metric=metric)
        for j in range(len(close_record_index)):
            closest_dict[i*index+j] = indexes_b[close_record_index[j]]
        
    
    dict_full_name = metric + 'from' + str(above) +'_dict.pkl'
    with open(dict_full_name, 'wb') as f:
        pickle.dump(closest_dict, f)

    closest_dict

In [9]:
dist_dict(2,'cosine')
dist_dict(3,'cosine')
dist_dict(4,'cosine')
dist_dict(5,'cosine')

In [10]:
dist_dict(2,'euclidean')
dist_dict(3,'euclidean')
dist_dict(4,'euclidean')
dist_dict(5,'euclidean')

In [ ]:
dist_dict(2,'hamming')
dist_dict(3,'hamming')
dist_dict(4,'hamming')

In [ ]:
dist_dict(2,'manhattan')
dist_dict(3,'manhattan')
dist_dict(4,'manhattan')

## use the closes neighbour to improve results

In [18]:
history_without_recs = ['user_clicks',  '61108514445dc5a3d61c81143f57fc213d1feeaf861f90594866145966fdc4f9_c_ctr',
       'de4d57b401ad9cbe913238688e5ab1f7ddcff1bd37538e9d529223dfe9d79576_c_ctr',
       '48509a07538677f82f9e1ade5c0d724fe1fd837343cd6bb0431b80a2520193f3_c_ctr',
       'a7c68f40778fdf5ab08ca67bcf62cc67427723c689fa7e75903ffe984169726b_c_ctr',
       '812df4136a77ca7a4ff72ab9e4a4f2deabeeb21817f6e25718582fd412d8e8e8_c_ctr',
       'f88b356a9face082abc26d4e75b829259cf770cc3fa2401bc152917d5aa66e7d_c_ctr',
       '6ad4d921f45ab9a3de6840e4ec06f625763eec88c8ed104bfd22a576c34ff039_c_ctr',
       'b5610de1f6ffecc890aa0a487b94a60ebc5f3190a6b8df7a773b87059826ab47_c_ctr',
       'bd9124bd238e8ea1d7472beb1f8bed2096e7d2ccaef939e348f80f95b01ba5bc_c_ctr',
       '68bf0109c6f8a87bfa96d1ecd9503326ba35cf4a6086988a1d2c8a20fbef00d1_c_ctr',
       '3be1d422a46085e17f4a19d3ed707c866927a6d354a562c45a78f1f03d7c4118_c_ctr',
       '11d40417959631d3d2420e8cd8709893c11cd7a4db737af63e8d56cfa7866f85_c_ctr',
       '1180412af43d78604a73f14d5a7ef83d3d3a74251ce8fb7b232043a511fbf161_c_ctr',
       '4263e28fc51808c7cc9a9540585fb69080542158e6dcc9910d05f8458f59ab49_c_ctr',
       'c513f6ef979bc9ca96e48c44a4e8fe756f4790570949e452e37df21e73567ef2_c_ctr',
       '0c350ae55158adeee57e63c575f9ff1975aeca281cf1e052a5cc3c18ada7c0ad_c_ctr',
       'd7efa2f7a9e1b512fd646d3e89e22e529f2b1e852a2864d654d9a867ca4a4624_c_ctr',
       '8e4b5bcfdfdc849677ee6d7a4b5f5d6c83eaf6619abd87bca4b3106b3722678c_c_ctr',
       '0209442e115ad7bc79fd281d91423a86b619e3c711fe574b7cc198d2e3c461c4_c_ctr',
       '28cec6db11ab7d5a0892c361b953500bc4b30cb96906b95fc346f6e0c3cd0e55_c_ctr',
       '8a91bb34721c6489a602f83428cff4adf368331fb8e7435dd8dc993cf482bfd5_c_ctr',
       'a17498f58d835f26f28be6e80da34f8e4d80ab429125db50197a0ba7fe619f49_c_ctr',
       '1cb542228c76558789d114d3cb273a75850cca54ec3ee9a41100f2dc56ee561e_c_ctr',
       'c0738849f57726a1e60cb17d93157b5b6ccf6f3bb7fdf6bbed57d84c9b23b31d_c_ctr',
       'a031eda273ac18861d11324af880859527405b4d475a9ac589bc4c8bfd598893_c_ctr',
       '0bcc639f6c41a8b9ea537cd08d9e8f8d6908c4dd79eeb431126b2781ba57771e_c_ctr',
       '7bc70a705d85bfa59f9784b64151fb3daf051eb1744830baa8d22bbcb32956f3_c_ctr',
       '33445260df1bc49114eeeda3d7e30ea3b4ac318f5e6bb7ad339e6a5a03be9336_c_ctr',
       '4b19c051cd2a88afdf08fff3eb4bfa99adf8d603da2a1c45e0576c1739eaea15_c_ctr',
       'a941e049e9b1a5f8f3e2c1d39ee03082c184e89224e519c0bef6d8694743159a_c_ctr',
       'a0da63f7deb716b7c1ee936a1b68b25dcba9773f0fb4de5f063dc52a053945b1_c_ctr',
       '790fad4744a9da8d301e23c83b386f1baabd229611dbe5265df62a97b5357393_c_ctr',
       '33297ca1e5b9d63741369135c744705be15edc2d9beb1629c1a45b1f61641a59_c_ctr',
       'ba5285161ba6eed0085fb13784ce5c92f70ebc268b94fd66aa1d68a32884204d_c_ctr',
       '516d189fdc3b2d3c12953d7e007b7151ed1538e23112ef5c1b6b2dbf8af7319e_c_ctr',
       '259aa8ef98a8b91de574cd904138ef643240c23080cf24da4793a6f10a43fa9d_c_ctr',
       '0a976f55a9c56ea0081016a41c4ba048732ffbfed98de816184703b7f93c1499_c_ctr',
       '0720c198df96f189a141cec90f859b927a91275264c2cbe16eb3966a58f727f9_c_ctr',
       'cb69f81bd35ddfa40a1d1608edd4b41d644dcf27a99fe3f9e2c094ba50ea54b1_c_ctr',
       '4a8a398e6be840d4f07f10e17b1750ded029c71311c5232bfa36d354d611dba9_c_ctr',
       '1a1d1484d3cbc721070469e555f6a0cc89c03d60181e1f13694014e9249d19b6_c_ctr',
       'f2bd5a01dfff120bb2e4be9b5a74970d991e5f6aca2cddf6680a54c98c7c17dc_c_ctr',
       'ca9942acaf5f9ae0a98c58ee0a1830f5c7047c465003fc24ac46df377eab8544_c_ctr',
       '2960f16ec1aa689ae008674ff33d51ef5d9b674bd90d160c34b888e5139e2009_c_ctr',
       '2096303a8868b9fa3a2ed62ce9fa0445269109f0f1750312ed419a3f74f8ac78_c_ctr',
       '19fba0e995b9794fc2c26217bf3b725c2f0d9eeda16719fe75e3ba23ca73bfc4_c_ctr',
       'd90d402744080c11741e5e10c35250c6f0a352d54fc2ec7718c208feddd7a2a2_c_ctr',
       '6fda2b8252b6d5ea9d19934d61b30d7a4a2922108fc1ecb83416001760139a39_c_ctr',
       'b537c6ef04866119578ef5f82075a5240805f9f54733b46639227110fefb413f_c_ctr',
       'b34564f1c4cd1d98dc26aaa4f888e3020656033add4dd0620b26e820493bf5c2_c_ctr',
       '61a78304b3c41ee12b310b29a17756d6d98ec5500f25df5c8d300156f0f23973_c_ctr',
       '16627e06ee5f755933fb2687ad25c3d81cb59cd8f1de7d96258509b1539cb2a8_c_ctr',
       'e2ddc2cd2e8ae62ed9e842999a8f2b74579198b3a293750ec07e36aa10491de2_c_ctr',
       'fe9bbd400bb6cb314531e3462507661401959afc69aae96bc6aec2c213b83bc1_c_ctr',
       'dd73a2f7c7982c61006be12e1bbb3e8c9ea6b6e8baf7cc5e307514015fc2fd23_c_ctr',
       'eab762a03fd979a04cc4706e6536d382bc89d2d1356afcd054a16b2235ecd471_c_ctr',
       '62484e22a6a5ade1ba25cb1b7c55c4b8861de24caddab73c9409742734008b26_c_ctr',
       '309d267f086d5fe5433d5bcc11cf83c14ffb5518dca95fc59b17363bb421e28e_c_ctr',
       '80f189984e5ca70287d13342f6daa0db45cba3c131c4e46dc81360f3a4c4f690_c_ctr',
       'd34a569ab7aaa54dacd715ae64953455d86b768846cd0085ef4e9e7471489b7b_c_ctr',
       '8517b64a7dfada982abdf62810e1c248d645151dc3d2352075c5cad87b3a2221_c_ctr',
       'c1f026582fe6e8cb620d0c85a72fe421ddded756662a8ec00ed4c297ad10676b_c_ctr',
       'c0674314c7945c60ae576e6b0b5469e0ba8b471652ee28610602be3c6aaeb7b4_c_ctr',
       '4ea140588150773ce3aace786aeef7f4049ce100fa649c94fbbddb960f1da942_c_ctr',
       'a172ddf41dd327dcac2c4211c298221f360f2d7488f9f1225070ece19b262991_c_ctr',
       'dd18a0f5cc4ab574f29f5eef2ca858e612473ac78cd6e260ea97ec4b3f2e1803_c_ctr',
       '124962dc269749b652c6e23e063617da6c15a4061ba980fc8f0b1ba77292415c_c_ctr',
       '2792b084bca1f0b04d2bc17e7f40cbab6661df68409cba8efe5be1498aa67d35_c_ctr',
       '114049bf85580c9d5a699a09bb8971425d0c9f641b6bfd326758333672ec73a5_c_ctr',
       '508592b6d926af8803391e1b90f21f9497c765349bb4e6e7bca62d81b5c1022d_c_ctr',
       '-6597379137822394620_i_clicks', '-3537769870962055354_i_clicks',
       '7641699375018129861_i_clicks', '2782413262714958813_i_clicks',
       '1071255553578813935_i_clicks', '1790838827304051171_i_clicks',
       '-277516354583730580_i_clicks', '-1844325170782087031_i_clicks',
       '6519237256383913780_i_clicks', '-6590137372813932048_i_clicks',
       '2050764513447150597_i_clicks', '6401048192163613996_i_clicks',
       '-5665231208787600560_i_clicks', '6052825694508831155_i_clicks',
       '-4658420179322495763_i_clicks', '-8304589174089261041_i_clicks',
       '2981167624921350096_i_clicks', '4147553327480165223_i_clicks',
       '-6613091900670437351_i_clicks', '5513023590059056719_i_clicks',
       '-2376436660574205512_i_clicks', '-4661562476615102361_i_clicks',
       '8006569233415963933_i_clicks', '-8846315789198768218_i_clicks',
       '-3473024805566572926_i_clicks', '-7191473408733797623_i_clicks',
       '7933936839897102492_i_clicks', '-5565074785503409293_i_clicks',
       '-6686171651625460591_i_clicks', '-8923508522781337425_i_clicks',
       '3978293437662790475_i_clicks', '-8349331379575229064_i_clicks',
       '5840779184213337588_i_clicks', '8173322118079640705_i_clicks',
       '-8549946995770995994_i_clicks', '-6755671448084645333_i_clicks',
       '-5598468937246360216_i_clicks', '821420889748784509_i_clicks',
       '1732518774282955304_i_clicks', '1286672594045415480_i_clicks',
       '-6004628949937626045_i_clicks', '1167157754418956910_i_clicks',
       '6285130451169023451_i_clicks', '1676633042548565843_i_clicks',
       '-1530069705287674516_i_clicks', '4015076361409988181_i_clicks',
       '-3435898480642252744_i_clicks', '-3596267220299612976_i_clicks',
       '-2940233538738748639_i_clicks', '-8245990465161984931_i_clicks',
       '-7439800813611300949_i_clicks', '7745802355884387631_i_clicks',
       '7151011376039939031_i_clicks', '-8244270229711223302_i_clicks',
       '81463566110175119_i_clicks', '7918195863372175678_i_clicks',
       '2507367679844346558_i_clicks', '7276627987286151880_i_clicks',
       '7496376258236356987_i_clicks', '-917443497655913392_i_clicks',
       '3703889155402940831_i_clicks', '3063910208987911824_i_clicks',
       '-4066981206407581872_i_clicks', '-7820953387091456096_i_clicks',
       '1019259011237425577_i_clicks', '6156748206734924010_i_clicks',
       '-867926204580748790_i_clicks', '6509554154811723184_i_clicks',
       '-2542437490169503076_i_clicks', '-4761594579425574936_i_clicks',
       '3601059902051216600_i_clicks', '7052693107312904282_i_clicks',
       '5246812338767375112_i_clicks', '-5621163822509059276_i_clicks',
       '4776874720734967109_i_clicks', '5590002166962482704_i_clicks',
       '8332108480529323818_i_clicks', '6388581769545739149_i_clicks',
       '-3371259035168698143_i_clicks', '5268119577047608705_i_clicks',
       '8977920698911593049_i_clicks', '8907799905198405812_i_clicks',
       '-4143464982107595149_i_clicks', '5789857031212100003_i_clicks',
       '-3938288290604949738_i_clicks', '8956548214050563956_i_clicks',
       '390462906908777733_i_clicks', '-127938762727969741_i_clicks',
       '-489809203262662140_i_clicks', '6060434067729885303_i_clicks',
       '-1422827765891273142_i_clicks', '-8955241989434981258_i_clicks',
       '964126797132705424_i_clicks', '5976994942744482604_i_clicks',
       '4923304358247540154_i_clicks', '-8029437237847644338_i_clicks',
       '1258821722976480117_i_clicks', '-8504951679593001779_i_clicks',
       '-5559330017833606475_i_clicks', '446851095707701277_i_clicks',
       '-2565032983111075329_i_clicks', '-3595323641892027824_i_clicks',
       '6541911725721430501_i_clicks', '-1715342302517748642_i_clicks',
       '2150205176954594318_i_clicks', '-6583257043694969040_i_clicks',
       '2277158520422351642_i_clicks', '-8977322542722062014_i_clicks',
       '3984870164980915490_i_clicks', '-7210248827359496438_i_clicks',
       '-7750113494543845989_i_clicks', '-5722753190249984610_i_clicks',
       '-4306681745995048802_i_clicks', '-9200345713046499318_i_clicks',
       '-3268549519706779451_i_clicks', '2470141795027120384_i_clicks',
       '2836056776580108007_i_clicks', '-4125475805878775176_i_clicks',
       '-4949128444307448220_i_clicks', '6712512344446708848_i_clicks',
       '-601946480702343428_i_clicks', '-8655025433408827997_i_clicks',
       '2220437081297852238_i_clicks', '-4083244903296105330_i_clicks',
       '6540618639587998193_i_clicks', '8579433685879267475_i_clicks',
       '-1058255037834804661_i_clicks', '-3193585481998003344_i_clicks',
       '-1395540213096405298_i_clicks', '7417192111309521036_i_clicks',
       '6984351696314919458_i_clicks', '3579901023400054608_i_clicks',
       '8490200088372093931_i_clicks', '9055377813200334935_i_clicks',
       '4436913629864704227_i_clicks', '6492059624586356549_i_clicks',
       '7288698517702336477_i_clicks', '-2002044985545801018_i_clicks',
       '4405515281669991437_i_clicks', '-750036118207051322_i_clicks',
       '7521853921239608992_i_clicks', '7557519904060801410_i_clicks',
       '8951753579110056782_i_clicks', '2839355173630047798_i_clicks',
       '7017123631563780887_i_clicks', '936630936040113192_i_clicks',
       '6915057301283748335_i_clicks', '-7956584263689547569_i_clicks',
       '-4798498070973557588_i_clicks', '-4537147676852755308_i_clicks',
       '7164253633056214_i_clicks', '5531386583286579610_i_clicks',
       '6709311094223257746_i_clicks', '3747692144586145998_i_clicks',
       '1597082942030427249_i_clicks', '27834901668906764_i_clicks',
       '-8742516677870340136_i_clicks', '101315062624410900_i_clicks',
       '-5827419750254943422_i_clicks', '-6987840845420875926_i_clicks',
       '4779478477400736725_i_clicks', '-4484468456254590098_i_clicks',
       '-62418374600764546_i_clicks', '8611949271977874056_i_clicks',
       '-4865872832970702614_i_clicks', '6923996694200037766_i_clicks',
       '-4352693544295965547_i_clicks', '-8837312109174350409_i_clicks',
       '6710410937413896675_i_clicks', '3779093080491101819_i_clicks',
       '-1546233820172493230_i_clicks', '-6922482799228753992_i_clicks',
       '-3165286775812642602_i_clicks', '8677353483444746886_i_clicks',
       '-8749875857714603232_i_clicks', '-3659437482324795455_i_clicks',
       '7731610252078196101_i_clicks', '-7630439413240589360_i_clicks',
       '2149269691410270673_i_clicks', '-4228920969204288365_i_clicks' ]

In [19]:
def evaluate_with_replacement(ls , ts1 , ts2):
    test_set_a_replace1 = ts2.reindex(ls)
    print('evaluation after changing the all row:')
    evaluate(test_set_a_replace1)
    print()
    
    ts1.loc[:, history_without_recs] = ts2.reindex(ls).loc[:, history_without_recs].values
    print('evaluation after replacing the history:')
    evaluate(ts1)
    print()

    ts1['user_recs_v2'] =  ts2.reindex(ls)['user_recs_v2'].values
    print('evaluation after the replacing history and the recs:')
    evaluate(ts1)
    print()

## cosine 

In [12]:
#cosine - with neighbors with 2 or more clicks
cosine_similarity = pd.read_pickle("cosinefrom2_dict.pkl") 
cosine_similarity = list(cosine_similarity.values())
evaluate_with_replacement(cosine_similarity,test_set_a, test_set_b)

evaluation after changing the all row:
Accuracy: 0.4388808472320822
Recall: 0.7652642095914742
F-Measure: 0.5138924387646432
Precision: 0.3868282242031681

evaluation after replacing the history:
Accuracy: 0.7059980515628985
Recall: 0.8375570921086019
F-Measure: 0.6883033523721396
Precision: 0.5841984458394403

evaluation after the replacing history and the recs:
Accuracy: 0.744198482431797
Recall: 0.7689434788124841
F-Measure: 0.6997088545030142
Precision: 0.6419119739986364



In [28]:
# test set a before changing it
test_set_a = test_set.loc[test_set['user_clicks']<2] #0-1 clicks
del test_set_a ['is_click']

#cosine - with neighbors with 3 or more clicks
cosine_similarity = pd.read_pickle("cosinefrom3_dict.pkl") 
cosine_similarity = list(cosine_similarity.values())
evaluate_with_replacement(cosine_similarity,test_set_a, test_set_b)

evaluation after changing the all row:
Accuracy: 0.4511553863222174
Recall: 0.7194240040598833
F-Measure: 0.5039828465409782
Precision: 0.3878392011353727



C:\admin\anaconda\lib\site-packages\pandas\core\indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


evaluation after replacing the history:
Accuracy: 0.7416569091149363
Recall: 0.8097564070032987
F-Measure: 0.7084238055927464
Precision: 0.629631913188236



C:\Users\E56A~1\AppData\Local\Temp/ipykernel_1468/2815741389.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts1['user_recs_v2'] =  ts2.reindex(ls)['user_recs_v2'].values


evaluation after the replacing history and the recs:
Accuracy: 0.7585443884089506
Recall: 0.7311675336209084
F-Measure: 0.7012491206722816
Precision: 0.6736829031291781



In [29]:
# test set a before changing it
test_set_a = test_set.loc[test_set['user_clicks']<2] #0-1 clicks
del test_set_a ['is_click']

#cosine - with neighbors with 4 or more clicks
cosine_similarity = pd.read_pickle("cosinefrom4_dict.pkl") 
cosine_similarity = list(cosine_similarity.values())
evaluate_with_replacement(cosine_similarity,test_set_a, test_set_b)

evaluation after changing the all row:
Accuracy: 0.4527780594918697
Recall: 0.7158240294341538
F-Measure: 0.5034689689019766
Precision: 0.3882819489367559



C:\admin\anaconda\lib\site-packages\pandas\core\indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


evaluation after replacing the history:
Accuracy: 0.7020827378921844
Recall: 0.8498556838365897
F-Measure: 0.6885924193605382
Precision: 0.578768880176692



C:\Users\E56A~1\AppData\Local\Temp/ipykernel_1468/2815741389.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts1['user_recs_v2'] =  ts2.reindex(ls)['user_recs_v2'].values


evaluation after the replacing history and the recs:
Accuracy: 0.7176056965662637
Recall: 0.7635990230905861
F-Measure: 0.677003437779012
Precision: 0.6080480381880864



In [30]:
# test set a before changing it
test_set_a = test_set.loc[test_set['user_clicks']<2] #0-1 clicks
del test_set_a ['is_click']

#cosine - with neighbors with 5 or more clicks
cosine_similarity = pd.read_pickle("cosinefrom5_dict.pkl") 
cosine_similarity = list(cosine_similarity.values())
evaluate_with_replacement(cosine_similarity,test_set_a, test_set_b)

evaluation after changing the all row:
Accuracy: 0.4548217671771326
Recall: 0.7060866531337224
F-Measure: 0.5009803452728229
Precision: 0.3882114441416894



C:\admin\anaconda\lib\site-packages\pandas\core\indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


evaluation after replacing the history:
Accuracy: 0.7087977774294767
Recall: 0.8408081705150977
F-Measure: 0.6911800173388174
Precision: 0.5867614019943115



C:\Users\E56A~1\AppData\Local\Temp/ipykernel_1468/2815741389.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts1['user_recs_v2'] =  ts2.reindex(ls)['user_recs_v2'].values


evaluation after the replacing history and the recs:
Accuracy: 0.717700967150088
Recall: 0.7567558995178889
F-Measure: 0.6751058434048152
Precision: 0.6093591372584074



## jaccard

In [31]:
# test set a before changing it
test_set_a = test_set.loc[test_set['user_clicks']<2] #0-1 clicks
del test_set_a ['is_click']

# with neighbors with 2 or more clicks
jaccard_similarity = pd.read_pickle("hamming_from_2_dict.pkl") 
jaccard_similarity = list(jaccard_similarity.values())
evaluate_with_replacement(jaccard_similarity,test_set_a, test_set_b)

evaluation after changing the all row:
Accuracy: 0.40332033350850827
Recall: 0.923504503933012
F-Measure: 0.5453974988468284
Precision: 0.38696419256340314



C:\admin\anaconda\lib\site-packages\pandas\core\indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


evaluation after replacing the history:
Accuracy: 0.7104020111312921
Recall: 0.8181695635625476
F-Measure: 0.6865140355429582
Precision: 0.5913561284258548



C:\Users\E56A~1\AppData\Local\Temp/ipykernel_1468/2815741389.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts1['user_recs_v2'] =  ts2.reindex(ls)['user_recs_v2'].values


evaluation after the replacing history and the recs:
Accuracy: 0.7454677324679077
Recall: 0.8297465744734839
F-Measure: 0.7164639991235997
Precision: 0.6303978504988192



In [32]:
# test set a before changing it
test_set_a = test_set.loc[test_set['user_clicks']<2] #0-1 clicks
del test_set_a ['is_click']

# with neighbors with 3 or more clicks
jaccard_similarity = pd.read_pickle("hamming_from_3_dict.pkl") 
jaccard_similarity = list(jaccard_similarity.values())
evaluate_with_replacement(jaccard_similarity,test_set_a, test_set_b)

evaluation after changing the all row:
Accuracy: 0.4171714470987034
Recall: 0.9277546942400406
F-Measure: 0.552350064204245
Precision: 0.3932330877608088



C:\admin\anaconda\lib\site-packages\pandas\core\indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


evaluation after replacing the history:
Accuracy: 0.6967660246658615
Recall: 0.817225957878711
F-Measure: 0.6762753737782692
Precision: 0.5767932796436107



C:\Users\E56A~1\AppData\Local\Temp/ipykernel_1468/2815741389.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts1['user_recs_v2'] =  ts2.reindex(ls)['user_recs_v2'].values


evaluation after the replacing history and the recs:
Accuracy: 0.7381687764491117
Recall: 0.822388036031464
F-Measure: 0.7088506371679602
Precision: 0.622859751007441



In [33]:
# test set a before changing it
test_set_a = test_set.loc[test_set['user_clicks']<2] #0-1 clicks
del test_set_a ['is_click']

# with neighbors with 4 or more clicks
jaccard_similarity = pd.read_pickle("hamming_from_4_dict.pkl") 
jaccard_similarity = list(jaccard_similarity.values())
evaluate_with_replacement(jaccard_similarity,test_set_a, test_set_b)

evaluation after changing the all row:
Accuracy: 0.397690149328957
Recall: 0.9573870844963207
F-Measure: 0.5519943492105581
Precision: 0.3877899077883661



C:\admin\anaconda\lib\site-packages\pandas\core\indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


evaluation after replacing the history:
Accuracy: 0.7374711499159468
Recall: 0.8033890510022836
F-Measure: 0.7034486110436092
Precision: 0.6256221209538982



C:\Users\E56A~1\AppData\Local\Temp/ipykernel_1468/2815741389.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts1['user_recs_v2'] =  ts2.reindex(ls)['user_recs_v2'].values


evaluation after the replacing history and the recs:
Accuracy: 0.763602949085556
Recall: 0.8339809058614565
F-Measure: 0.7322354153395227
Precision: 0.6526163602405078



## euclidean

In [34]:
# test set a before changing it
test_set_a = test_set.loc[test_set['user_clicks']<2] #0-1 clicks
del test_set_a ['is_click']

# with neighbors with 2 or more clicks
euclidean_similarity = pd.read_pickle("euclideanfrom2_dict.pkl") 
euclidean_similarity = list(euclidean_similarity.values())
evaluate_with_replacement(euclidean_similarity,test_set_a, test_set_b)

evaluation after changing the all row:
Accuracy: 0.4028685665465028
Recall: 0.9356286475513829
F-Measure: 0.5484419737478154
Precision: 0.3879135763505339



C:\admin\anaconda\lib\site-packages\pandas\core\indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


evaluation after replacing the history:
Accuracy: 0.7230576325567244
Recall: 0.8072348388733824
F-Measure: 0.693195514064511
Precision: 0.6073887569657411



C:\Users\E56A~1\AppData\Local\Temp/ipykernel_1468/2815741389.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts1['user_recs_v2'] =  ts2.reindex(ls)['user_recs_v2'].values


evaluation after the replacing history and the recs:
Accuracy: 0.7525177556709046
Recall: 0.8210400279116975
F-Measure: 0.7200136293782639
Precision: 0.641125187304182



In [35]:
# test set a before changing it
test_set_a = test_set.loc[test_set['user_clicks']<2] #0-1 clicks
del test_set_a ['is_click']

#- with neighbors with 3 or more clicks
euclidean_similarity = pd.read_pickle("euclideanfrom3_dict.pkl") 
euclidean_similarity = list(euclidean_similarity.values())
evaluate_with_replacement(euclidean_similarity,test_set_a, test_set_b)

evaluation after changing the all row:
Accuracy: 0.40159624326575266
Recall: 0.9353511164679016
F-Measure: 0.5478413123037769
Precision: 0.38736043609615134



C:\admin\anaconda\lib\site-packages\pandas\core\indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


evaluation after replacing the history:
Accuracy: 0.721914385550833
Recall: 0.8076709591474245
F-Measure: 0.6924337185588035
Precision: 0.6059742515825044



C:\Users\E56A~1\AppData\Local\Temp/ipykernel_1468/2815741389.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts1['user_recs_v2'] =  ts2.reindex(ls)['user_recs_v2'].values


evaluation after the replacing history and the recs:
Accuracy: 0.7515927090344172
Recall: 0.8206594138543517
F-Measure: 0.7191672544203128
Precision: 0.6400155837409636



In [36]:
# test set a before changing it
test_set_a = test_set.loc[test_set['user_clicks']<2] #0-1 clicks
del test_set_a ['is_click']

#with neighbors with 4 or more clicks
euclidean_similarity = pd.read_pickle("euclideanfrom4_dict.pkl") 
euclidean_similarity = list(euclidean_similarity.values())
evaluate_with_replacement(euclidean_similarity,test_set_a, test_set_b)

evaluation after changing the all row:
Accuracy: 0.4016822941156585
Recall: 0.9358189545800558
F-Measure: 0.5480007986589959
Precision: 0.38743967696398673



C:\admin\anaconda\lib\site-packages\pandas\core\indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


evaluation after replacing the history:
Accuracy: 0.7226365980411139
Recall: 0.8085035523978685
F-Measure: 0.6932064247471742
Precision: 0.6066891582323295



C:\Users\E56A~1\AppData\Local\Temp/ipykernel_1468/2815741389.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts1['user_recs_v2'] =  ts2.reindex(ls)['user_recs_v2'].values


evaluation after the replacing history and the recs:
Accuracy: 0.7521397465802471
Recall: 0.819501712763258
F-Measure: 0.7193273684613484
Precision: 0.6409757065685916



In [37]:
# test set a before changing it
test_set_a = test_set.loc[test_set['user_clicks']<2] #0-1 clicks
del test_set_a ['is_click']

# with neighbors with 5 or more clicks
euclidean_similarity = pd.read_pickle("euclideanfrom5_dict.pkl") 
euclidean_similarity = list(euclidean_similarity.values())
evaluate_with_replacement(euclidean_similarity,test_set_a, test_set_b)

evaluation after changing the all row:
Accuracy: 0.39996435036218186
Recall: 0.9362312864755138
F-Measure: 0.5473996546008785
Precision: 0.38676856453076425



C:\admin\anaconda\lib\site-packages\pandas\core\indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


evaluation after replacing the history:
Accuracy: 0.7243422488160325
Recall: 0.8098594899771632
F-Measure: 0.6948721263292535
Precision: 0.6084778075662794



C:\Users\E56A~1\AppData\Local\Temp/ipykernel_1468/2815741389.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts1['user_recs_v2'] =  ts2.reindex(ls)['user_recs_v2'].values


evaluation after the replacing history and the recs:
Accuracy: 0.7516295879700912
Recall: 0.8211113930474498
F-Measure: 0.7193084214072609
Precision: 0.6399644024745225



## manhattan

In [38]:
# test set a before changing it
test_set_a = test_set.loc[test_set['user_clicks']<2] #0-1 clicks
del test_set_a ['is_click']

#  neighbors with 2 or more clicks
manhattan_similarity = pd.read_pickle("manhattan_from_2_dict.pkl") 
manhattan_similarity = list(manhattan_similarity.values())
evaluate_with_replacement(manhattan_similarity,test_set_a, test_set_b)

evaluation after changing the all row:
Accuracy: 0.40263807319854084
Recall: 0.938736995686374
F-Measure: 0.5491676852674379
Precision: 0.38810628288557053



C:\admin\anaconda\lib\site-packages\pandas\core\indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


evaluation after replacing the history:
Accuracy: 0.7223108341093276
Recall: 0.8076630296878965
F-Measure: 0.6927353784315926
Precision: 0.6064409342867519



C:\Users\E56A~1\AppData\Local\Temp/ipykernel_1468/2815741389.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts1['user_recs_v2'] =  ts2.reindex(ls)['user_recs_v2'].values


evaluation after the replacing history and the recs:
Accuracy: 0.7520383295071438
Recall: 0.8214761481857397
F-Measure: 0.7197304432402389
Precision: 0.6404109589041096



In [39]:
# test set a before changing it
test_set_a = test_set.loc[test_set['user_clicks']<2] #0-1 clicks
del test_set_a ['is_click']

#  neighbors with 3 or more clicks
manhattan_similarity = pd.read_pickle("manhattan_from_3_dict.pkl") 
manhattan_similarity = list(manhattan_similarity.values())
evaluate_with_replacement(manhattan_similarity,test_set_a, test_set_b)

evaluation after changing the all row:
Accuracy: 0.40008420690312213
Recall: 0.9343678634864248
F-Measure: 0.5469555045999314
Precision: 0.3866434355763804



C:\admin\anaconda\lib\site-packages\pandas\core\indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


evaluation after replacing the history:
Accuracy: 0.7222309297487007
Recall: 0.8075758056330881
F-Measure: 0.6926511444437266
Precision: 0.6063610004703472



C:\Users\E56A~1\AppData\Local\Temp/ipykernel_1468/2815741389.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts1['user_recs_v2'] =  ts2.reindex(ls)['user_recs_v2'].values


evaluation after the replacing history and the recs:
Accuracy: 0.7518539348287742
Recall: 0.8215237249429079
F-Measure: 0.7195921542479301
Precision: 0.6401631240731587



In [40]:
# test set a before changing it
test_set_a = test_set.loc[test_set['user_clicks']<2] #0-1 clicks
del test_set_a ['is_click']

#  neighbors with 4 or more clicks
manhattan_similarity = pd.read_pickle("manhattan_from_4_dict.pkl") 
manhattan_similarity = list(manhattan_similarity.values())
evaluate_with_replacement(manhattan_similarity,test_set_a, test_set_b)

evaluation after changing the all row:
Accuracy: 0.4002501621136547
Recall: 0.9360171910682568
F-Measure: 0.5474610307996977
Precision: 0.38686640928665533



C:\admin\anaconda\lib\site-packages\pandas\core\indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


evaluation after replacing the history:
Accuracy: 0.7220772675167262
Recall: 0.8085035523978685
F-Measure: 0.6927778174121899
Precision: 0.6060328687330976



C:\Users\E56A~1\AppData\Local\Temp/ipykernel_1468/2815741389.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts1['user_recs_v2'] =  ts2.reindex(ls)['user_recs_v2'].values


evaluation after the replacing history and the recs:
Accuracy: 0.7516295879700912
Recall: 0.8215792311596042
F-Measure: 0.7194234114130379
Precision: 0.6398624071340789



## The results

| | Accuracy | Recall | F-measure | Precision | 
|----------:|:----------|:----------|:----------|:----------|
| original data 0-1 clicks | 0.7889633638506526 | 0.7600862725196651 | 0.7362749202130724 | 0.7139101356232638 |
| cosine | | | | |
| all row | 0.4388808472320822 | 0.7652642095914742 | 0.5138924387646432 | 0.3868282242031681 |
| history | 0.7059980515628985 | 0.8375570921086019 | 0.6883033523721396 | 0.5841984458394403 |
|  history with recs | 0.744198482431797 | 0.7689434788124841 | 0.6997088545030142 | 0.6419119739986364 |
| all row from 3 | 0.4511553863222174 | 0.7194240040598833 | 0.5039828465409782 | 0.3878392011353727 |
| history from 3 | 0.7416569091149363 | 0.8097564070032987 | 0.7084238055927464 | 0.629631913188236 |
| history with recs from 3 | 0.7585443884089506 | 0.7311675336209084 | 0.7012491206722816 | 0.6736829031291781 |
| all row from 4 | 0.4527780594918697 | 0.7158240294341538 | 0.5034689689019766 | 0.3882819489367559 |
| history from 4 | 0.7020827378921844 | 0.8498556838365897 | 0.6885924193605382 | 0.578768880176692 |
| history with recs from 4 | 0.7176056965662637 | 0.7635990230905861 | 0.677003437779012 | 0.6080480381880864 |
| all row from 5 | 0.4548217671771326 | 0.7060866531337224 | 0.5009803452728229 | 0.3882114441416894 |
| history from 5 | 0.7087977774294767 | 0.8408081705150977 | 0.6911800173388174 | 0.5867614019943115 |
| history with recs from 5 | 0.717700967150088 | 0.7567558995178889 | 0.6751058434048152 | 0.6093591372584074 |
| jaccard | | | | |
| all row | 0.40332033350850827 | 0.923504503933012 | 0.5453974988468284 | 0.38696419256340314 |
| history | 0.7104020111312921 | 0.8181695635625476 | 0.6865140355429582 | 0.5913561284258548 |
| history with recs | 0.7454677324679077 | 0.8297465744734839 | 0.7164639991235997 | 0.6303978504988192 |
| all row from 3 | 0.4171714470987034 | 0.9277546942400406 | 0.552350064204245 | 0.3932330877608088 |
| history from 3 | 0.6967660246658615 | 0.817225957878711 | 0.6762753737782692 | 0.5767932796436107 |
| history with recs from 3 | 0.7381687764491117 | 0.822388036031464 | 0.7088506371679602 | 0.622859751007441 |
| all row from 4 | 0.397690149328957 | 0.9573870844963207 | 0.5519943492105581 | 0.3877899077883661 |
| history from 4 | 0.7374711499159468 | 0.8033890510022836 | 0.7034486110436092 | 0.6256221209538982 |
| history with recs from 4 | 0.763602949085556 | 0.8339809058614565 | 0.7322354153395227 | 0.6526163602405078 |
| euclidean | | | | |
| all row | 0.4028685665465028 | 0.9356286475513829 | 0.5484419737478154 | 0.3879135763505339 |
| history | 0.7230576325567244 | 0.8072348388733824 | 0.693195514064511 | 0.6073887569657411 |
| history with recs | 0.7525177556709046 | 0.8210400279116975 | 0.7200136293782639 | 0.641125187304182 |
| all row from 3 | 0.40159624326575266 | 0.9353511164679016 | 0.5478413123037769 | 0.38736043609615134 |
| history from 3 | 0.721914385550833 | 0.8076709591474245 | 0.6924337185588035 | 0.6059742515825044 |
| history with recs from 3 | 0.7515927090344172 | 0.8206594138543517 | 0.7191672544203128 | 0.6400155837409636 |
| all row from 4 | 0.4016822941156585 | 0.9358189545800558 | 0.5480007986589959 | 0.38743967696398673 |
| history from 4 | 0.7226365980411139 | 0.8085035523978685 | 0.6932064247471742 | 0.6066891582323295 |
| history with recs from 4 | 0.7521397465802471 | 0.819501712763258 | 0.7193273684613484 | 0.6409757065685916 |
| all row from 5 | 0.39996435036218186 | 0.9362312864755138 | 0.5473996546008785 | 0.38676856453076425 |
| history from 5 | 0.7243422488160325 | 0.8098594899771632 | 0.6948721263292535 | 0.5031336017889639 |
| history with recs from 5 | 0.7516295879700912 | 0.8211113930474498 | 0.7193084214072609 | 0.6399644024745225 |
| Manhattan | | | | |
| all row | 0.40263807319854084  | 0.938736995686374 | 0.5491676852674379 | 0.38810628288557053 |
| history | 0.7223108341093276 | 0.8076630296878965 | 0.6927353784315926 | 0.6064409342867519 |
| history with recs | 0.7520383295071438 | 0.8214761481857397 | 0.7197304432402389 | 0.6404109589041096 |
| all row from 3 | 0.40008420690312213 | 0.9343678634864248 | 0.5469555045999314 | 0.3866434355763804 |
| history from 3 | 0.7222309297487007 | 0.8075758056330881 | 0.6926511444437266 | 0.6063610004703472 |
| history with recs from 3 | 0.7518539348287742 | 0.8215237249429079 | 0.7195921542479301 | 0.6401631240731587 |
| all row from 4 | 0.4002501621136547 | 0.9360171910682568 | 0.5474610307996977 | 0.38686640928665533 |
| history from 4 | 0.7220772675167262 | 0.8085035523978685 | 0.6927778174121899 | 0.6060328687330976 |
| history with recs from 4 | 0.7516295879700912 | 0.8215792311596042 | 0.7194234114130379 | 0.6398624071340789 |
